<a href="https://colab.research.google.com/github/biondo999/Cfd/blob/main/TEMPLATE_CFDlab05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
    import firedrake
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
    import firedrake

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from firedrake import *
import matplotlib.pyplot as plt
import numpy as np

from firedrake.petsc import PETSc

### Useful classes and functions from Lab 4.

In [ ]:
# Object representing S = B*A^{-1}*B'.
class SchurComplement(object):
    def __init__(self, ksp_A, B):
        self.ksp_A = ksp_A
        self.B = B
        self.tmp_u1 = B.createVecRight()    # vector to which we can apply B*tmp_u1
        self.tmp_u2 = B.createVecRight()    # vector to which we can apply B*tmp_u2
        self.tmp_p  = B.createVecLeft()     # vector in which we can store tmp_p=B*x
    def mult(self, mat, in_vec, out_vec):
        # Implements out_vec = B*A^{-1}*B'*in_vec

        # tmp_u1 = B'*in_vec
        self.B.multTranspose(in_vec, self.tmp_u1)
        # tmp_u2=A^{-1}*tmp_u1   ->   solve A*tmp_u2 = tmp_u1
        self.ksp_A.solve(self.tmp_u1, self.tmp_u2)
        # out_vec = B*tmp_u2
        self.B.mult(self.tmp_u2, out_vec)

        # print("A solved in", self.ksp_A.getIterationNumber(), "iterations.")


# Create a preconditioner (see doc above on MatrixFreePC)
# based on a PETSC.Mat and possibly using lumping or its diagonal component
class MyMatrixPC(object):
    # Inputs:
    #   - mat:      the preconditioning matrix
    #   - lumping:  if True, use the lumped version of mat
    #   - use_diag: if True, use the diagonal component of mat
    def __init__(self, mat, lumping, use_diag):
        self.mat = mat
        self.lumping = lumping
        self.use_diag = use_diag
        self.vec = self.mat.createVecLeft()

    def setUp(self, pc):
        # check flags
        if self.lumping and self.use_diag:
            raise(BaseException('Error in MyMatrixPC: you can (possibly) either use lumping or diag, not both!'))

        if self.lumping:
            # Compute mat*ones and store it in vec.
            tmp = self.mat.createVecRight()
            tmp.set(1.0)
            self.mat.mult(tmp, self.vec)

        elif self.use_diag:
            self.vec = self.mat.getDiagonal()

        else: # use the full matrix mat
            S, P_S = pc.getOperators()
            self.pc = PETSc.PC().create()
            self.pc.setOptionsPrefix('pc_MyMatrixPC_')
            self.pc.setOperators(self.mat)
            self.pc.setFromOptions()

        # # Print mat and vec. Uncomment only for small mesh.
        #     np.set_printoptions(precision=5)
        #     np.set_printoptions(linewidth=400)
        #     np.set_printoptions(suppress=True)
        # with dim = self.mat.getSizes()[0][0]:
        #     print('Preconditioner: mat =', self.mat.getValues(range(dim), range(dim)))
        # print('vec =', self.vec.getArray())

    # Implement "pc^{-1}*in_vec = out_vec" depending on the strategy.
    def apply(self, mat, in_vec, out_vec):
        if self.lumping or self.use_diag:
            # out_vec[i] = in_vec[i]/self.vec[i]
            out_vec.pointwiseDivide(in_vec, self.vec)
        else: # use the full matrix mat
            self.pc.apply(in_vec, out_vec)


# Create a preconditioner based on PETSc.Mat P and and set it into ksp.
# Inputs:   ksp solver
#           preconditioner P as PETSc Matrix
def set_KSP_PC(ksp, P, lumping=False, use_diag=False):
    if lumping and use_diag:
        raise(BaseException('Error in set_KSP_PC: you can (possibly) either use lumping or diag, not both!'))
    MpPC = MyMatrixPC(P, lumping, use_diag)
    pc = ksp.pc
    pc.setType(pc.Type.PYTHON)
    pc.setPythonContext(MpPC)

def create_ksp_schur(ksp_A, B, useNullSpace):
    S = PETSc.Mat().create()
    S.setSizes( B.getSize()[0], B.getSize()[0] )    # B is m-by-n  ->  S is m-by-m (square)
    # print("Size   ->   A:", A_fd.M.handle.getSize(), "  B:", B.getSize(), "  S:", S.getSize())
    S.setType(S.Type.PYTHON)            # i.e. user-defined
    Sctx = SchurComplement(ksp_A, B)    # build the matrix "context" [ https://www.firedrakeproject.org/petsc-interface.html#building-an-operator ]
    S.setPythonContext(Sctx)            # set context into S
    S.setUp()

    if useNullSpace:
        null_vec = S.createVecLeft()
        null_vec.array[:] = np.sqrt(1.0/null_vec.getSize())
        null_vec.normalize()
        nsp = PETSc.NullSpace().create(vectors=[null_vec])
        S.setNullSpace(nsp)

    ksp_S = PETSc.KSP().create()
    ksp_S.setType('cg')     # S is positive definite
    ksp_S.setOperators(S)

    return ksp_S

---
---
# Exercise 1
## Solve unsteady Navier-Stokes problem by Schur-complement method.

\begin{equation*}
\begin{cases}
\frac{\partial\boldsymbol{u}}{\partial t} + (\boldsymbol{u}\cdot\nabla)\boldsymbol{u} - \frac{1}{\rm Re}\Delta \boldsymbol{u} + \nabla  p  = \boldsymbol{f} & {\rm in} \ \Omega=(0,1)^2
\quad\forall t\in(0,1), \\
{\rm div}\,\boldsymbol{u} = 0 & {\rm in} \ \Omega
\quad\forall t\in(0,1), \\
\boldsymbol{u} = \boldsymbol{u}_\text{ex} & {\rm on} \ \partial\Omega
\quad\forall t\in(0,1), \\
\boldsymbol{u} = \boldsymbol{0} & {\rm in} \ \Omega,
\quad{\rm for} \ t=0.
\end{cases}
\end{equation*}

### Point 3. Assemble the matrices defining the problem (penalty method for Dirichlet BCs).

In [ ]:
def ex1_assemble_Euler(V, Q, Re, n, dt, t, u_old, u_ex_fun, p_ex_fun, f_fun):
    # Trial and test functions.

    u=TrialFunction(V)
    v=TestFunction(V)
    p=TrialFunction(Q)
    q=TestFunction(Q)


    # Data.
    x = SpatialCoordinate(mesh)
    u_ex=u_ex_fun(x,t)
    f=f_fun(x,t)


    # Variational forms with the penalty method for Dirichlet boundary conditions.
    eps = 1e-30
    a= 1.0/dt *inner(u,v)*dx +1.0/Re *inner(grad(u),grad(v))*dx \
        +inner(dot(grad(u),u_old),v)*dx \
        +1/eps *inner(u,v)*ds
    b=-div(u)*q*dx
    L=inner(f,v)*dx +1.0/dt *inner(u_old,v)*dx \
       +1.0/eps *inner(u_ex,v)*ds


    # Assemble matrices: all PETSc types.
    A=assemble(a).M.handle
    B=assemble(b).M.handle
    F=assemble(L)

    return V, Q, A, B, F


def ex1_assemble_finalize(A, B, P, F, setupSchurNullSpace=False):
    ksp_UU = PETSc.KSP().create()
    ksp_UU.setType('preonly')   # direct solver
    ksp_UU.pc.setType('lu')
    ksp_UU.setOperators(A)
    ksp_UU.setFromOptions()
    ksp_UU.setUp()

    # Set verbose to True for detailed logging of ksp_S.
    # These options are stored GLOBALLY: petsc_options is just a proxy.
    petsc_options = PETSc.Options()
    verbose = False
    if verbose:
        petsc_options['ksp_view'] = ''
        petsc_options['ksp_monitor_true_residual'] = ''
    else:
        del petsc_options['ksp_view']
        del petsc_options['ksp_monitor_true_residual']

    ksp_S = create_ksp_schur(ksp_UU, B, setupSchurNullSpace)

   # ... set preconditioner in kps_S ...

    # Finalize setup of ksp_S.
    ksp_S.setFromOptions()
    ksp_S.setUp()

    return ksp_UU, ksp_S, F

In [ ]:
# solve problem

def ex1_solve(V, Q, ksp_A, B, ksp_S, vecF):
    # Actual functions to store solution and to create temporary vectors.
    uh = Function(V)
    ph = Function(Q)
    tmp_u_fun = Function(V)
    tmp_p_fun = Function(Q)
    rhs_p_fun = Function(Q)

    with uh.vector().dat.vec_wo as vecU,\
      ph.vector().dat.vec_wo as vecP,\
      tmp_u_fun.vector().dat.vec_wo as tmp_u,\
      tmp_p_fun.vector().dat.vec_wo as tmp_p,\
      rhs_p_fun.vector().dat.vec_wo as rhs_p:

      #computing chi (rhs)
      ksp_A.solve(vecF,tmp_u)
      B.mult(tmp_u,rhs_p)

      ksp_S.solve(rhs_p,vecP)

      B.multTranspose(vecP, tmp_u)
      ksp_A.solve(vecF - tmp_u, vecU)




    return uh, ph

In [ ]:
def lab4_ex1_solve(V, Q, ksp_A, B, ksp_S, vecF):
    # Actual functions to store solution and to create temporary vectors.
    uh = Function(V)
    ph = Function(Q)
    tmp_u_fun = Function(V)
    tmp_p_fun = Function(Q)
    rhs_p_fun = Function(Q)
    # Extract the dof vectors as PETSc.Vec and give them aliases.
    # Temporary vectors are in read/write mode,
    # while momentum rhs is in read-only mode.
    with uh.vector().dat.vec_wo as vecU,\
        ph.vector().dat.vec_wo as vecP,\
        tmp_u_fun.vector().dat.vec_wo as tmp_u,\
        tmp_p_fun.vector().dat.vec_wo as tmp_p,\
        rhs_p_fun.vector().dat.vec_wo as rhs_p:

        # rhs_p = B*(A^{-1}*F)
        ksp_A.solve(vecF, tmp_u)
        B.mult(tmp_u, rhs_p)

        # solve S*P = rhs_p and store it into the dof array of ph
        ksp_S.solve(rhs_p, vecP)

        # reconstruct velocity U = A^{-1}*(F-B'*P) and store it into the dof array of uh
        B.multTranspose(vecP, tmp_u)
        ksp_A.solve(vecF - tmp_u, vecU)

    print("Schur-based solver:")
    print("     # iterations =", ksp_S.getIterationNumber())
    print("     last iter residual norm =", ksp_S.getResidualNorm())
    print("     (only to check convergence of kps_S: it is NOT a measure of the actual error)")

    return uh, ph

In [ ]:
# Data
Re = 10

#lamda functions that also depend on time

u_ex_fun = lambda x, t: as_vector((
    -cos(x[0]) * sin(x[1]) * sin(2*t),
    sin(x[0]) * cos(x[1]) * sin(2*t)))
p_ex_fun = lambda x, t: -0.25*(cos(2*x[0])+cos(2*x[1]))*sin(2*t)*sin(2*t)
f_fun = lambda x, t: as_vector((
    -2 * cos(x[0]) * sin(x[1]) * ( cos(2*t) + sin(2*t)/Re ),
    2 * sin(x[0]) * cos(x[1]) * ( cos(2*t) + sin(2*t)/Re )))

# Parameters
n = 10
t0 = 0
T = 0.3
dt = 0.1

# Problem setup: mesh, FE spaces
mesh = UnitSquareMesh(n, n, 'crossed')
V=VectorFunctionSpace(mesh,'P',2)
Q=FunctionSpace(mesh,'P',1)


# Preconditioner for Schur complement

p=TrialFunction(Q)
q=TestFunction(Q)
form=Re*p*q*dx +dt*inner(grad(p),grad(q))*dx
P=assemble(form).M.handle


In [ ]:
# Initial condition.
u0 = interpolate(Constant((0., 0.)), V)
print('t =', t0, ' :  ||u||_{L^2} =', norm(u0,'L2'))

# Solve time-dependent problem.

#arange is floating pint version of range
for t in np.arange(t0+dt, T+0.1*dt, dt):  # T+0.1*dt to include also T: range/arang exclude the upper bound of the range

   # Assemble time-dependent terms and finalize assembly
   V,Q,A,B,F=ex1_assemble_Euler(V,Q,Re,n,dt,t,u0,u_ex_fun,p_ex_fun,f_fun)
   ksp_UU,ksp_S,F=ex1_assemble_finalize(A,B,P,F,True)


   # Advance in time
   with F.dat.vec_ro as vecF:
      uh, ph = lab4_ex1_solve(V,Q,ksp_UU,B,ksp_S,vecF)
      print('t =', t, ' :  ||u||_{L^2} =', norm(uh,'L2'))

      fig, ax = plt.subplots()
      col = tripcolor(ph, axes=ax)
      plt.colorbar(col)
      plt.title('pressure')
      fig, ax = plt.subplots()
      # triplot(mesh, axes=ax)
      col = quiver(uh, axes=ax)
      plt.colorbar(col)
      plt.title('velocity')

      u0=uh

